In [5]:
import os 

os.getcwd()

os.chdir('/Users/matth/Dropbox/Seasonal Macro')

In [34]:
from calendar import month
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm 
import os
from datetime import datetime
from pyspark.sql.functions import date_format
from pandas import ExcelWriter
import re as re


In [7]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

## BM/BEA Data

In [8]:
#Map Months to Quarters

"""
DateTime objects are reported as mm-dd-yyyy. All the BEA/BM data is reported at a quarterly level, measured in the first month of each quarter.
This function takes in a month (NUM) and returns the quarter it corresponds to.
"""

def month_to_quarter(num):
    if num == 1:
        return num
    elif num == 4:
        return 2
    elif num == 7:
        return 3
    elif num == 10:
        return 4


#Lagged % Change Calculation, Quarterly
"""
This function takes in a series (corresponding to some index, such as "durable goods") and computes a % change from time period to time period.
"""

def perc_change_quart(series, index):
    lst=[]
    for j in range(len(series)):
        if j==0 or isinstance(series.loc[index[j]], str) or isinstance(series.loc[index[j-1]],str):
            lst.append(np.nan)
        else:
            if series.loc[index[j-1]] == 0:
                lst.append(np.nan)
            else:
                lst.append((series.loc[index[j]]-series.loc[index[j-1]])/series.loc[index[j-1]])
    return lst 

#Unique Column Names; from https://stackoverflow.com/questions/19071622/automatically-rename-columns-to-ensure-they-are-unique

"""
Some of the dataframes report multiple indices with the same name under different categories.
For instance, 'goods' and 'services' are reported a total of 3x (domestic, import, export) but all have the same name in the dataframe.
This function renames them as [col_name]_[number] where number is one of {1,2,3, ...}
"""

def unique_cols(df_columns):
    seen = set()
    for item in df_columns:
        fudge = 1
        newitem = item
        while newitem in seen:
            fudge += 1
            newitem = "{}_{}".format(item, fudge)
        yield newitem
        seen.add(newitem)

Cleaning BEA and BM Dataframes

In [9]:
#Defining Data

"""
We use BEA data from 8 different tables. We analyze the BEA data by creating a LIST of DATAFRAMES, where each element of the list corresponds to one of the tables in the master sheet.

Each DATAFRAME reports years as columns and indices as rows. We take the transpose so each row is a time period and each column is an index.

Data is read in from 'bea_master_sheet.xlsx'.

Barsky Miron data is read in from 'data_final.dta'.
"""

sheet_names = ['Table 8.1.3', 'Table 8.1.4', 'Table 8.1.5', 'Table 8.1.6', 'Table 8.1.11', 'Table 8.2', 'Table 8.3', 'Table 8.4'] 

bea_data = []

bea_years = []

for i in range(len(sheet_names)):
    bea_data.append(pd.read_excel('Data/barsky_miron_updated/BEA NIPA/bea_master_sheet.xlsx', sheet_name=sheet_names[i], na_values='---', parse_dates=True, usecols=lambda x: x!= ('List', 'xq', '')))
    bea_data[i] = bea_data[i].transpose()
    bea_years.append(list(range(int(bea_data[i].index[2]), 2022)))
    
bm_data = pd.read_stata('Data/barsky_miron_updated/data_final.dta')


In [10]:
#Column Names, BEA

for i in range(len(sheet_names)): #range(len(sheet_names)) because each nested list = 1 sheet
    bea_data[i].index=range(len(bea_data[i].index))
    bea_data[i].columns = [j for j in bea_data[i].iloc[1]]
    bea_data[i].drop(index=[0,1], axis=0, inplace=True)
    bea_data[i].index=range(len(bea_data[i].index))
    if 'Residuals' in bea_data[i].columns:
        bea_data[i].drop('Residuals')


In [11]:
#Reindexing BEA Dataframe to DateTime Objects

bea_indexnames=[]
for i in range(len(sheet_names)):
    bea_indexnames.append([])

for i in range(len(sheet_names)):
    j=1
    while j<=len(bea_data[i].index):
        if j == len(bea_data[i].index):
            bea_indexnames[i].append('10/01/2021') #All BEA data ends with Q4 of 2021
        elif j%4 == 0:
            bea_indexnames[i].append(f'10/01/{bea_years[i][j//4]-1}')
        else:
            bea_indexnames[i].append(f'0{3*(j%4)-2}/01/{bea_years[i][j//4]}')
        j+=1

for i in range(len(sheet_names)):
    bea_data[i].index = bea_indexnames[i]
    bea_data[i].index = pd.to_numeric(bea_data[i].index, errors='ignore')
    bea_data[i].index = pd.to_datetime(bea_data[i].index, format='%m/%d/%Y')
    bea_data[i].pop(bea_data[i].columns[0])


In [12]:
#Reindexing BM Dataframe to DateTime Objects

bm_indexnames=[]
for i in range(len(bm_data.columns)):
    bm_indexnames.append([])

for i in range(len(bm_data.columns)):
    j=1
    while j<=len(bm_data.index):
        if j == len(bm_data.index):
            bm_indexnames[i].append('04/01/2019') #All BM data ends with Q2 of 2019
        elif j%4 == 0:
            bm_indexnames[i].append(f"10/01/{bm_data['year'][j]-1}")
        else:
            bm_indexnames[i].append(f"0{3*(j%4)-2}/01/{bm_data['year'][j]}")
        j+=1

for i in range(len(bm_data.columns)):
    bm_data.index = bm_indexnames[i]
    bm_data.index = pd.to_numeric(bm_data.index, errors='ignore')
    bm_data.index = pd.to_datetime(bm_data.index, format='%m/%d/%Y')


In [13]:
#Removing whitespace from BEA/BM column names

#Some names were read in like '                 Services'. This code block just turns that name into 'Services'. 

for i in range(len(sheet_names)):
    bea_data[i].columns=[j.strip() for j in bea_data[i].columns if isinstance(j,str)]
    
bm_data.columns = [i.strip() for i in bm_data.columns if isinstance(i,str)]

In [14]:
#BEA data contains three columns named goods/services. Need to rename them:
for k in range(len(sheet_names)):
    bea_data[k].columns = unique_cols(list(bea_data[k].columns))

In [15]:
#Dropping Unnecessary BM Columns

#BM dataset has some unnecessary/poorly named variables. We are interested in comparing the BM data where we know the BEA data can match each category.

bm_droplists = ['var19', 'var20', 'var22', 'var23']+list(bm_data.columns[28:]) #Info on wages/employment starts from 28th column

bm_data = bm_data.drop(columns=bm_droplists)


Quarterly % Changes, BM

In [16]:
#This creates quarterly % change variables for each index in the BM dataset

quarter_indices_bm = bm_data.index 
quarter_names_bm=list(quarter_indices_bm)

quarterly_per_change_names_bm=[f'Quarterly % Change, {i}' for i in bm_data.columns[2:24]] #omit year/month + quarter dummies

for i in range(0,22):
    bm_data[quarterly_per_change_names_bm[i]]=perc_change_quart(bm_data[bm_data.columns[i+2]], list(quarter_indices_bm))



Quarterly % Changes, BEA

In [17]:
#This creates quarterly % change variables for each table in the BEA data. 
#This is very similar to the computations for the BM data, except we do it for each dataframe in our list of dataframes.


quarterly_per_change_names_bea=[]
quarter_indices_bea = []
quarter_names_bea=[]

for i in range(len(sheet_names)):
    quarterly_per_change_names_bea.append([])
    quarter_indices_bea.append([])
    quarter_names_bea.append([])

for i in range(len(sheet_names)):
    quarterly_per_change_names_bea[i]=[f'Quarterly % Change, {i}' for i in bea_data[i].columns] 
    quarter_indices_bea[i]=bea_data[i].index
    quarter_names_bea[i]=list(quarter_indices_bea[i])

for i in range(len(sheet_names)):
    for j in range(len(bea_data[i].columns)):
        quarterly_name=quarterly_per_change_names_bea[i][j]
        column_name=bea_data[i].columns[j]
        bea_data[i][quarterly_name]=perc_change_quart(bea_data[i][column_name], list(bea_data[i].index))

In [18]:
#Defining Quarter + Year + Quarter Dummy Variables for BM/BEA 

#BEA Quarter + Year
for i in range(len(sheet_names)):
    bea_data[i]['Year'] = pd.DatetimeIndex(bea_data[i].index).year
    bea_data[i]['Quarter'] = pd.DatetimeIndex(bea_data[i].index).month.map(month_to_quarter)

#BM Quarter + Year were already defined in the original dataset

#BEA Defining Quarterly Dummies 

quarter_dummies_bea=[]

for i in range(len(sheet_names)):
    quarter_dummies_bea.append([])
    quarter_dummies_bea[i] = pd.get_dummies(bea_data[i]['Quarter'])
    bea_data[i]=bea_data[i].join(quarter_dummies_bea[i])

#BM Defining Quarterly Dummies

quarter_dummies_bm = pd.get_dummies(bm_data['quarter'])
bm_data=bm_data.join(quarter_dummies_bm)


## Regressions

BM, Quarterly

In [19]:
#Quarterly % Change, 5 lags (BM)

bm_summary_quarterly_5lags = [] #this is a list of regression table outputs
bm_data_quarterly_5lags = [] #this is a list of point estimates, standard errors, and t-values. This is used to make plots

for i in range(22):
    temp_df = bm_data.copy()
    indices=np.where(bm_data[quarterly_per_change_names_bm[i]].notnull())[0] #This makes sure we don't run into any nan issues when running the regression
    index_names_temp = [k for k in quarter_names_bm if quarter_names_bm.index(k) in indices] #This makes sure the monthly dummies match up with the corresponding % change
    temp_df['y'] = bm_data[quarterly_per_change_names_bm[i]].dropna()
    temp_df['x_1'] = bm_data[1].loc[index_names_temp]
    temp_df['x_2'] = bm_data[2].loc[index_names_temp]
    temp_df['x_3'] = bm_data[3].loc[index_names_temp]
    temp_df['x_4'] = bm_data[4].loc[index_names_temp]
    ols = sm.ols(formula = 'y~x_1+x_2+x_3+x_4-1', data=temp_df).fit(cov_type='HAC',cov_kwds={'maxlags':5})
    bm_summary_quarterly_5lags.append(ols.summary(yname = temp_df[quarterly_per_change_names_bm], xname = ['Q1', 'Q2', 'Q3', 'Q4']))
    df = pd.DataFrame
    df = pd.concat((ols.params.rename('coefficient'), ols.bse.rename('se'), ols.tvalues.rename('t')), axis=1)
    bm_data_quarterly_5lags.append(df)

In [20]:
#Quarterly % Change, 10 lags (BM)

bm_summary_quarterly_10lags = []
bm_data_quarterly_10lags = []

for i in range(22):
    temp_df = bm_data.copy()
    indices=np.where(bm_data[quarterly_per_change_names_bm[i]].notnull())[0]
    index_names_temp = [k for k in quarter_names_bm if quarter_names_bm.index(k) in indices]
    temp_df['y'] = bm_data[quarterly_per_change_names_bm[i]].dropna()
    temp_df['x_1'] = bm_data[1].loc[index_names_temp]
    temp_df['x_2'] = bm_data[2].loc[index_names_temp]
    temp_df['x_3'] = bm_data[3].loc[index_names_temp]
    temp_df['x_4'] = bm_data[4].loc[index_names_temp]
    ols = sm.ols(formula = 'y~x_1+x_2+x_3+x_4-1', data=temp_df).fit(cov_type='HAC',cov_kwds={'maxlags':10})
    bm_summary_quarterly_10lags.append(ols.summary(yname = temp_df[quarterly_per_change_names_bm], xname = ['Q1', 'Q2', 'Q3', 'Q4']))
    df = pd.DataFrame
    df = pd.concat((ols.params.rename('coefficient'), ols.bse.rename('se'), ols.tvalues.rename('t')), axis=1)
    bm_data_quarterly_10lags.append(df)

BEA, Quarterly

In [21]:
#Quarterly % Change, 5 lags (BEA)

bea_summary_quarterly_5lags = []
bea_data_quarterly_5lags = []

for i in range(len(sheet_names)):
    bea_summary_quarterly_5lags.append({})
    bea_data_quarterly_5lags.append({})

for i in range(len(sheet_names)): 
    for j in range(len(quarterly_per_change_names_bea[i])):
        try:
            temp_df=pd.DataFrame()
            quarterly_name = quarterly_per_change_names_bea[i][j]
            indices=(np.where(bea_data[i][quarterly_name].notnull())[0]).tolist() 
            temp_df['y'] = bea_data[i][quarterly_name].dropna()
            temp_df['x_1'] = bea_data[i][1].iloc[indices] 
            temp_df['x_2'] = bea_data[i][2].iloc[indices]
            temp_df['x_3'] = bea_data[i][3].iloc[indices]
            temp_df['x_4'] = bea_data[i][4].iloc[indices]
            ols = sm.ols(formula = 'y~x_1+x_2+x_3+x_4-1', data=temp_df).fit(cov_type='HAC',cov_kwds={'maxlags':5})
            bea_summary_quarterly_5lags[i][quarterly_name]=ols.summary(yname = quarterly_name, xname = ['Q1', 'Q2', 'Q3', 'Q4'])
            df = pd.DataFrame
            df = pd.concat((ols.params.rename('coefficient'), ols.bse.rename('se'), ols.tvalues.rename('t')), axis=1)
            bea_data_quarterly_5lags[i][quarterly_name]=df
        except ValueError:
            continue

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/scipy/stats/_stats_py.py:1477: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=11
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [22]:
#Quarterly % Change, 10 lags (BEA)

bea_summary_quarterly_10lags = []
bea_data_quarterly_10lags = []

for i in range(len(sheet_names)):
    bea_summary_quarterly_10lags.append({})
    bea_data_quarterly_10lags.append({})

for i in range(len(sheet_names)): 
    for j in range(len(quarterly_per_change_names_bea[i])):
        try:
            temp_df=pd.DataFrame()
            quarterly_name = quarterly_per_change_names_bea[i][j]
            indices=(np.where(bea_data[i][quarterly_name].notnull())[0]).tolist() 
            temp_df['y'] = bea_data[i][quarterly_name].dropna()
            temp_df['x_1'] = bea_data[i][1].iloc[indices] 
            temp_df['x_2'] = bea_data[i][2].iloc[indices]
            temp_df['x_3'] = bea_data[i][3].iloc[indices]
            temp_df['x_4'] = bea_data[i][4].iloc[indices]
            ols = sm.ols(formula = 'y~x_1+x_2+x_3+x_4-1', data=temp_df).fit(cov_type='HAC',cov_kwds={'maxlags':10})
            bea_summary_quarterly_10lags[i][quarterly_name]=ols.summary(yname = quarterly_name, xname = ['Q1', 'Q2', 'Q3', 'Q4'])
            df = pd.DataFrame
            df = pd.concat((ols.params.rename('coefficient'), ols.bse.rename('se'), ols.tvalues.rename('t')), axis=1)
            bea_data_quarterly_10lags[i][quarterly_name]=df
        except ValueError:
            continue

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/scipy/stats/_stats_py.py:1477: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=11
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


# Regressions, Corrected Years

In [23]:
#Setting up Years

"""
BEA and BM data use different years. Only want to report years with both BEA and BM
BM data begins from 1947, ends mid 2019. 
BEA data either starts in 1947, 2002, or 2003, ends in 2021.

Here, I regress from years 1947-mid 2019, 2002-mid 2019, or 2003-mid 2019 based on the BEA index.
"""

#Years from 1947-2019 are just the bm_data
yrs_1947_2019 = [k for k in bm_data.index]
yrs_2002_2019 = [i for i in bea_data[0].index if i in bm_data.index] #bea_data[0] starts from 2002
yrs_2003_2019 = [j for j in bea_data[4].index if j in bm_data.index] #bea_data[4] starts from 2003

BM, Quarterly

In [24]:
#Quarterly % Change, 5 lags (BM), 1947

#This regression is the same as was run above in the non-corrected section.

bm_summary_quarterly_5lags_1947 = []
bm_data_quarterly_5lags_1947 = []

for i in range(22): #first 22 columns are the columns that we have BEA data on
    temp_df = bm_data.copy()
    indices=np.where(bm_data[quarterly_per_change_names_bm[i]].notnull())[0]
    index_names_temp = [k for k in quarter_names_bm if quarter_names_bm.index(k) in indices]
    temp_df['y'] = bm_data[quarterly_per_change_names_bm[i]].dropna()
    temp_df['x_1'] = bm_data[1].loc[index_names_temp]
    temp_df['x_2'] = bm_data[2].loc[index_names_temp]
    temp_df['x_3'] = bm_data[3].loc[index_names_temp]
    temp_df['x_4'] = bm_data[4].loc[index_names_temp]
    ols = sm.ols(formula = 'y~x_1+x_2+x_3+x_4-1', data=temp_df).fit(cov_type='HAC',cov_kwds={'maxlags':5})
    bm_summary_quarterly_5lags_1947.append(ols.summary(yname = temp_df[quarterly_per_change_names_bm], xname = ['Q1', 'Q2', 'Q3', 'Q4']))
    df = pd.DataFrame
    df = pd.concat((ols.params.rename('coefficient'), ols.bse.rename('se'), ols.tvalues.rename('t')), axis=1)
    bm_data_quarterly_5lags_1947.append(df)


In [25]:
#Quarterly % Change, 5 lags (BM), 2002

bm_summary_quarterly_5lags_2002 = []
bm_data_quarterly_5lags_2002 = []

for i in range(22): #first 22 columns are the columns that we have BEA data on
    temp_df = bm_data.loc[yrs_2002_2019].copy()
    indices=np.where(bm_data.loc[yrs_2002_2019][quarterly_per_change_names_bm[i]].notnull())[0]
    index_names_temp = [k for k in yrs_2002_2019 if yrs_2002_2019.index(k) in indices]
    temp_df['y'] = bm_data.loc[yrs_2002_2019][quarterly_per_change_names_bm[i]].dropna()
    temp_df['x_1'] = bm_data.loc[yrs_2002_2019][1].loc[index_names_temp]
    temp_df['x_2'] = bm_data.loc[yrs_2002_2019][2].loc[index_names_temp]
    temp_df['x_3'] = bm_data.loc[yrs_2002_2019][3].loc[index_names_temp]
    temp_df['x_4'] = bm_data.loc[yrs_2002_2019][4].loc[index_names_temp]
    ols = sm.ols(formula = 'y~x_1+x_2+x_3+x_4-1', data=temp_df).fit(cov_type='HAC',cov_kwds={'maxlags':5})
    bm_summary_quarterly_5lags_2002.append(ols.summary(yname = temp_df[quarterly_per_change_names_bm], xname = ['Q1', 'Q2', 'Q3', 'Q4']))
    df = pd.DataFrame
    df = pd.concat((ols.params.rename('coefficient'), ols.bse.rename('se'), ols.tvalues.rename('t')), axis=1)
    bm_data_quarterly_5lags_2002.append(df)



In [26]:
#Quarterly % Change, 5 lags (BM), 2003

bm_summary_quarterly_5lags_2003 = []
bm_data_quarterly_5lags_2003 = []

for i in range(22): #first 22 columns are the columns that we have BEA data on
    temp_df = bm_data.loc[yrs_2003_2019].copy()
    indices=np.where(bm_data.loc[yrs_2003_2019][quarterly_per_change_names_bm[i]].notnull())[0]
    index_names_temp = [k for k in yrs_2003_2019 if yrs_2003_2019.index(k) in indices]
    temp_df['y'] = bm_data.loc[yrs_2003_2019][quarterly_per_change_names_bm[i]].dropna()
    temp_df['x_1'] = bm_data.loc[yrs_2003_2019][1].loc[index_names_temp]
    temp_df['x_2'] = bm_data.loc[yrs_2003_2019][2].loc[index_names_temp]
    temp_df['x_3'] = bm_data.loc[yrs_2003_2019][3].loc[index_names_temp]
    temp_df['x_4'] = bm_data.loc[yrs_2003_2019][4].loc[index_names_temp]
    ols = sm.ols(formula = 'y~x_1+x_2+x_3+x_4-1', data=temp_df).fit(cov_type='HAC',cov_kwds={'maxlags':5})
    bm_summary_quarterly_5lags_2003.append(ols.summary(yname = temp_df[quarterly_per_change_names_bm], xname = ['Q1', 'Q2', 'Q3', 'Q4']))
    df = pd.DataFrame
    df = pd.concat((ols.params.rename('coefficient'), ols.bse.rename('se'), ols.tvalues.rename('t')), axis=1)
    bm_data_quarterly_5lags_2003.append(df)



In [27]:
#Quarterly % Change, 10 lags (BM), 1947

#This regression is the same as was run above in the non-corrected section.

bm_summary_quarterly_10lags_1947 = []
bm_data_quarterly_10lags_1947 = []

for i in range(22): #first 22 columns are the columns that we have BEA data on
    temp_df = bm_data.copy()
    indices=np.where(bm_data[quarterly_per_change_names_bm[i]].notnull())[0]
    index_names_temp = [k for k in quarter_names_bm if quarter_names_bm.index(k) in indices]
    temp_df['y'] = bm_data[quarterly_per_change_names_bm[i]].dropna()
    temp_df['x_1'] = bm_data[1].loc[index_names_temp]
    temp_df['x_2'] = bm_data[2].loc[index_names_temp]
    temp_df['x_3'] = bm_data[3].loc[index_names_temp]
    temp_df['x_4'] = bm_data[4].loc[index_names_temp]
    ols = sm.ols(formula = 'y~x_1+x_2+x_3+x_4-1', data=temp_df).fit(cov_type='HAC',cov_kwds={'maxlags':10})
    bm_summary_quarterly_10lags_1947.append(ols.summary(yname = temp_df[quarterly_per_change_names_bm], xname = ['Q1', 'Q2', 'Q3', 'Q4']))
    df = pd.DataFrame
    df = pd.concat((ols.params.rename('coefficient'), ols.bse.rename('se'), ols.tvalues.rename('t')), axis=1)
    bm_data_quarterly_10lags_1947.append(df)


In [28]:
#Quarterly % Change, 10 lags (BM), 2002

bm_summary_quarterly_10lags_2002 = []
bm_data_quarterly_10lags_2002 = []

for i in range(22): #first 22 columns are the columns that we have BEA data on
    temp_df = bm_data.loc[yrs_2002_2019].copy()
    indices=np.where(bm_data.loc[yrs_2002_2019][quarterly_per_change_names_bm[i]].notnull())[0]
    index_names_temp = [k for k in yrs_2002_2019 if yrs_2002_2019.index(k) in indices]
    temp_df['y'] = bm_data.loc[yrs_2002_2019][quarterly_per_change_names_bm[i]].dropna()
    temp_df['x_1'] = bm_data.loc[yrs_2002_2019][1].loc[index_names_temp]
    temp_df['x_2'] = bm_data.loc[yrs_2002_2019][2].loc[index_names_temp]
    temp_df['x_3'] = bm_data.loc[yrs_2002_2019][3].loc[index_names_temp]
    temp_df['x_4'] = bm_data.loc[yrs_2002_2019][4].loc[index_names_temp]
    ols = sm.ols(formula = 'y~x_1+x_2+x_3+x_4-1', data=temp_df).fit(cov_type='HAC',cov_kwds={'maxlags':10})
    bm_summary_quarterly_10lags_2002.append(ols.summary(yname = temp_df[quarterly_per_change_names_bm], xname = ['Q1', 'Q2', 'Q3', 'Q4']))
    df = pd.DataFrame
    df = pd.concat((ols.params.rename('coefficient'), ols.bse.rename('se'), ols.tvalues.rename('t')), axis=1)
    bm_data_quarterly_10lags_2002.append(df)



In [29]:
#Quarterly % Change, 10 lags (BM), 2003

bm_summary_quarterly_10lags_2003 = []
bm_data_quarterly_10lags_2003 = []

for i in range(22): #first 22 columns are the columns that we have BEA data on
    temp_df = bm_data.loc[yrs_2003_2019].copy()
    indices=np.where(bm_data.loc[yrs_2003_2019][quarterly_per_change_names_bm[i]].notnull())[0]
    index_names_temp = [k for k in yrs_2003_2019 if yrs_2003_2019.index(k) in indices]
    temp_df['y'] = bm_data.loc[yrs_2003_2019][quarterly_per_change_names_bm[i]].dropna()
    temp_df['x_1'] = bm_data.loc[yrs_2003_2019][1].loc[index_names_temp]
    temp_df['x_2'] = bm_data.loc[yrs_2003_2019][2].loc[index_names_temp]
    temp_df['x_3'] = bm_data.loc[yrs_2003_2019][3].loc[index_names_temp]
    temp_df['x_4'] = bm_data.loc[yrs_2003_2019][4].loc[index_names_temp]
    ols = sm.ols(formula = 'y~x_1+x_2+x_3+x_4-1', data=temp_df).fit(cov_type='HAC',cov_kwds={'maxlags':10})
    bm_summary_quarterly_10lags_2003.append(ols.summary(yname = temp_df[quarterly_per_change_names_bm], xname = ['Q1', 'Q2', 'Q3', 'Q4']))
    df = pd.DataFrame
    df = pd.concat((ols.params.rename('coefficient'), ols.bse.rename('se'), ols.tvalues.rename('t')), axis=1)
    bm_data_quarterly_10lags_2003.append(df)



BEA, Quarterly

In [30]:
#Defining BEA dataframe to be restricted to end mid 2019

bea_data_corrected=[]

for i in bea_data:
    if i.index[0] == pd.Timestamp('2002-01-01 00:00:00'):
        bea_data_corrected.append(i.loc[yrs_2002_2019])
    elif i.index[0] == pd.Timestamp('2003-01-01 00:00:00'):
        bea_data_corrected.append(i.loc[yrs_2003_2019])
    elif i.index[0] == pd.Timestamp('1947-01-01 00:00:00'):
        bea_data_corrected.append(i.loc[yrs_1947_2019])


In [31]:
#Quarterly % Change, 5 lags (BEA), Corrected

bea_summary_quarterly_5lags_corrected = []
bea_data_quarterly_5lags_corrected = []

for i in range(len(sheet_names)):
    bea_summary_quarterly_5lags_corrected.append({})
    bea_data_quarterly_5lags_corrected.append({})

for i in range(len(sheet_names)): 
    for j in range(len(quarterly_per_change_names_bea[i])):
        try:
            temp_df=pd.DataFrame()
            quarterly_name = quarterly_per_change_names_bea[i][j]
            indices=(np.where(bea_data_corrected[i][quarterly_name].notnull())[0]).tolist() 
            temp_df['y'] = bea_data_corrected[i][quarterly_name].dropna()
            temp_df['x_1'] = bea_data_corrected[i][1].iloc[indices] 
            temp_df['x_2'] = bea_data_corrected[i][2].iloc[indices]
            temp_df['x_3'] = bea_data_corrected[i][3].iloc[indices]
            temp_df['x_4'] = bea_data_corrected[i][4].iloc[indices]
            ols = sm.ols(formula = 'y~x_1+x_2+x_3+x_4-1', data=temp_df).fit(cov_type='HAC',cov_kwds={'maxlags':5})
            bea_summary_quarterly_5lags_corrected[i][quarterly_name]=ols.summary(yname = quarterly_name, xname = ['Q1', 'Q2', 'Q3', 'Q4'])
            df = pd.DataFrame
            df = pd.concat((ols.params.rename('coefficient'), ols.bse.rename('se'), ols.tvalues.rename('t')), axis=1)
            bea_data_quarterly_5lags_corrected[i][quarterly_name]=df
        except ValueError:
            continue

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/statsmodels/stats/stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 4 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1918: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1736: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


In [32]:
#Quarterly % Change, 10 lags (BEA), Corrected

bea_summary_quarterly_10lags_corrected = []
bea_data_quarterly_10lags_corrected = []

for i in range(len(sheet_names)):
    bea_summary_quarterly_10lags_corrected.append({})
    bea_data_quarterly_10lags_corrected.append({})

for i in range(len(sheet_names)): 
    for j in range(len(quarterly_per_change_names_bea[i])):
        try:
            temp_df=pd.DataFrame()
            quarterly_name = quarterly_per_change_names_bea[i][j]
            indices=(np.where(bea_data_corrected[i][quarterly_name].notnull())[0]).tolist() 
            temp_df['y'] = bea_data_corrected[i][quarterly_name].dropna()
            temp_df['x_1'] = bea_data_corrected[i][1].iloc[indices] 
            temp_df['x_2'] = bea_data_corrected[i][2].iloc[indices]
            temp_df['x_3'] = bea_data_corrected[i][3].iloc[indices]
            temp_df['x_4'] = bea_data_corrected[i][4].iloc[indices]
            ols = sm.ols(formula = 'y~x_1+x_2+x_3+x_4-1', data=temp_df).fit(cov_type='HAC',cov_kwds={'maxlags':10})
            bea_summary_quarterly_10lags_corrected[i][quarterly_name]=ols.summary(yname = quarterly_name, xname = ['Q1', 'Q2', 'Q3', 'Q4'])
            df = pd.DataFrame
            df = pd.concat((ols.params.rename('coefficient'), ols.bse.rename('se'), ols.tvalues.rename('t')), axis=1)
            bea_data_quarterly_10lags_corrected[i][quarterly_name]=df
        except ValueError:
            continue

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/statsmodels/stats/stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 4 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1918: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1736: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


# Plots

BEA (5, 10 lags)

In [39]:
#Dictionary for Folder Names for BEA

bea_folders={0: 'Table 8.1.3', 1: 'Table 8.1.4', 2: 'Table 8.1.5', 3: 'Table 8.1.6', 4: 'Table 8.1.11', 5: 'Table 8.2', 6: 'Table 8.3', 7: 'Table 8.4'}

In [40]:
#Visualizations: BEA, 5 lags

quarter_list = ['Q1', 'Q2', 'Q3', 'Q4']

for i in range(len(sheet_names)):
    for j in bea_data_quarterly_5lags[i].keys():
        plot = plt.errorbar(x=quarter_list, y=bea_data_quarterly_5lags[i][j]['coefficient'], yerr = bea_data_quarterly_5lags[i][j]['se'])
        plt.savefig(f'Data Visualizations/BEA, BM/BEA, 5 lags/{bea_folders[i]}/{j[20:]}_quarterly_5lags.png', bbox_inches='tight') #string slicing is just to get rid of the "Quarterly % Change," section of the name
        plt.clf()

<Figure size 432x288 with 0 Axes>

In [41]:
#Visualizations: BEA, 10 lags

quarter_list = ['Q1', 'Q2', 'Q3', 'Q4']

for i in range(len(sheet_names)):
    for j in bea_data_quarterly_10lags[i].keys():
        plot = plt.errorbar(x=quarter_list, y=bea_data_quarterly_10lags[i][j]['coefficient'], yerr = bea_data_quarterly_10lags[i][j]['se'])
        plt.savefig(f'Data Visualizations/BEA, BM/BEA, 10 lags/{bea_folders[i]}/{j[20:]}_quarterly_10lags.png', bbox_inches='tight')
        plt.clf()

<Figure size 432x288 with 0 Axes>

BM (5,10 lags)

In [46]:
#Visualizations: BM, 5 lags 

quarter_list = ['Q1', 'Q2', 'Q3', 'Q4']

for i in range(22):
    plot = plt.errorbar(x=quarter_list, y=bm_data_quarterly_5lags[i]['coefficient'], yerr = bm_data_quarterly_5lags[i]['se'])
    plt.savefig(f'Data Visualizations/BEA, BM/BM, 5 lags/{bm_data.columns[i+2]}_quarterly_5lags.png', bbox_inches='tight')
    plt.clf()

<Figure size 432x288 with 0 Axes>

In [ ]:
#Visualizations, BM, 10 lags

quarter_list = ['Q1', 'Q2', 'Q3', 'Q4']

for i in range(22):
    plot = plt.errorbar(x=quarter_list, y=bm_data_quarterly_10lags[i]['coefficient'], yerr = bm_data_quarterly_10lags[i]['se'])
    plt.savefig(f'Data Visualizations/BEA, BM/BM, 10 lags/{bm_data.columns[i+2]}_quarterly_10lags.png', bbox_inches='tight')
    plt.clf()

BEA + BM (5,10 lags)

In [65]:
#Visualizations, BEA + BM (5 lags)

#List of Names to Compare

quarter_list=['Q1', 'Q2', 'Q3', 'Q4']
compare_names=[i for i in quarterly_per_change_names_bea[0][0:25] if i not in ['Quarterly % Change, Goods_2', 'Quarterly % Change, Goods_3', 'Quarterly % Change, Services_2', 'Quarterly % Change, Services_3', 'Quarterly % Change, Change in private inventories', 'Quarterly % Change, Net exports of goods and services']] #compare the 1st 26 observations, minus the import/export breakdown of goods + services. Only report aggregate goods/services
folder_names = {0: 'Table 8.1.3', 1: 'Table 8.1.4', 2: 'Table 8.1.5', 3: 'Table 8.1.6', 4: 'Table 8.1.11'}

for i in range(5): #Want Tables 8.1.3, 8.1.4, 8.1.5, 8.1.6, 8.1.11
    for j,k in zip(compare_names, range(22)): 
        y1, y2 = bea_data_quarterly_5lags[i][j]['coefficient'], bm_data_quarterly_5lags[k]['coefficient']
        yerr1, yerr2 = bea_data_quarterly_5lags[i][j]['se'], bm_data_quarterly_5lags[k]['se']
        er1 = plt.errorbar(quarter_list, y1, yerr1, label='BEA')
        er2 = plt.errorbar(quarter_list, y2, yerr2, label='BM')
        plt.legend(loc='best')
        plt.savefig(f'Data Visualizations/BEA, BM/BEA + BM, 5 lags/{folder_names[i]}/{j[20:]}_quarterly_5lags.png', bbox_inches='tight') 
        plt.clf()

<Figure size 432x288 with 0 Axes>

In [ ]:
#Visualizations, BEA + BM (10 lags)

#List of Names to Compare

quarter_list=['Q1', 'Q2', 'Q3', 'Q4']
compare_names=[i for i in quarterly_per_change_names_bea[0][0:25] if i not in ['Quarterly % Change, Goods_2', 'Quarterly % Change, Goods_3', 'Quarterly % Change, Services_2', 'Quarterly % Change, Services_3', 'Quarterly % Change, Change in private inventories', 'Quarterly % Change, Net exports of goods and services']] #compare the 1st 26 observations, minus the import/export breakdown of goods + services. Only report aggregate goods/services
folder_names = {0: 'Table 8.1.3', 1: 'Table 8.1.4', 2: 'Table 8.1.5', 3: 'Table 8.1.6', 4: 'Table 8.1.11'}

for i in range(5): #Want Tables 8.1.3, 8.1.4, 8.1.5, 8.1.6, 8.1.11
    for j,k in zip(compare_names, range(22)): #use compare_names[0] since for some reason it's a nested list
        y1, y2 = bea_data_quarterly_10lags[i][j]['coefficient'], bm_data_quarterly_10lags[k]['coefficient']
        yerr1, yerr2 = bea_data_quarterly_10lags[i][j]['se'], bm_data_quarterly_10lags[k]['se']
        er1 = plt.errorbar(quarter_list, y1, yerr1, label='BEA')
        er2 = plt.errorbar(quarter_list, y2, yerr2, label='BM')
        plt.legend(loc='best')
        plt.savefig(f'Data Visualizations/BEA, BM/BEA + BM, 10 lags/{folder_names[i]}/{j[20:]}_quarterly_10lags.png', bbox_inches='tight') 
        plt.clf()

# Plots, Corrected

BEA (5, 10 lags)

In [43]:
#Visualizations: BEA, 5 lags, Corrected

quarter_list = ['Q1', 'Q2', 'Q3', 'Q4']

for i in range(len(sheet_names)):
    for j in bea_data_quarterly_5lags[i].keys():
        plot = plt.errorbar(x=quarter_list, y=bea_data_quarterly_5lags_corrected[i][j]['coefficient'], yerr = bea_data_quarterly_5lags[i][j]['se'])
        plt.savefig(f'Data Visualizations/Corrected BEA, BM/BEA, 5 lags/{bea_folders[i]}/{j[20:]}_quarterly_5lags.png', bbox_inches='tight') #string slicing is just to get rid of the "Quarterly % Change," section of the name
        plt.clf()

<Figure size 432x288 with 0 Axes>

In [44]:
#Visualizations: BEA, 10 lags, Corrected

quarter_list = ['Q1', 'Q2', 'Q3', 'Q4']

for i in range(len(sheet_names)):
    for j in bea_data_quarterly_10lags[i].keys():
        plot = plt.errorbar(x=quarter_list, y=bea_data_quarterly_10lags_corrected[i][j]['coefficient'], yerr = bea_data_quarterly_10lags[i][j]['se'])
        plt.savefig(f'Data Visualizations/Corrected BEA, BM/BEA, 10 lags/{bea_folders[i]}/{j[20:]}_quarterly_5lags.png', bbox_inches='tight') #string slicing is just to get rid of the "Quarterly % Change," section of the name
        plt.clf()

<Figure size 432x288 with 0 Axes>

BM (5, 10 lags)

In [67]:
#Visualizations: BM, 5 lags, 1947-2019

quarter_list = ['Q1', 'Q2', 'Q3', 'Q4']

for i in range(22):
    plot = plt.errorbar(x=quarter_list, y=bm_data_quarterly_5lags_1947[i]['coefficient'], yerr = bm_data_quarterly_5lags_1947[i]['se'])
    plt.savefig(f'Data Visualizations/Corrected BEA, BM/BM, 5 lags, 1947/{bm_data.columns[i+2]}_quarterly_5lags.png', bbox_inches='tight') #i+2 since the first 2 columns are year+quarter variables
    plt.clf()

<Figure size 432x288 with 0 Axes>

In [69]:
#Visualizations: BM, 10 lags, 1947-2019

quarter_list = ['Q1', 'Q2', 'Q3', 'Q4']

for i in range(22):
    plot = plt.errorbar(x=quarter_list, y=bm_data_quarterly_10lags_1947[i]['coefficient'], yerr = bm_data_quarterly_10lags_1947[i]['se'])
    plt.savefig(f'Data Visualizations/Corrected BEA, BM/BM, 10 lags, 1947/{bm_data.columns[i+2]}_quarterly_10lags.png', bbox_inches='tight') #i+2 since the first 2 columns are year+quarter variables
    plt.clf()

<Figure size 432x288 with 0 Axes>

In [51]:
#Visualizations: BM, 5 lags, 2002-2019

quarter_list = ['Q1', 'Q2', 'Q3', 'Q4']

for i in range(22):
    plot = plt.errorbar(x=quarter_list, y=bm_data_quarterly_5lags_2002[i]['coefficient'], yerr = bm_data_quarterly_5lags_2002[i]['se'])
    plt.savefig(f'Data Visualizations/Corrected BEA, BM/BM, 5 lags, 2002/{bm_data.columns[i+2]}_quarterly_5lags.png', bbox_inches='tight') #i+2 since the first 2 columns are year+quarter variables
    plt.clf()

<Figure size 432x288 with 0 Axes>

In [52]:
#Visualizations: BM, 10 lags, 2002-2019

quarter_list = ['Q1', 'Q2', 'Q3', 'Q4']

for i in range(22):
    plot = plt.errorbar(x=quarter_list, y=bm_data_quarterly_10lags_2002[i]['coefficient'], yerr = bm_data_quarterly_10lags_2002[i]['se'])
    plt.savefig(f'Data Visualizations/Corrected BEA, BM/BM, 10 lags, 2002/{bm_data.columns[i+2]}_quarterly_10lags.png', bbox_inches='tight') #i+2 since the first 2 columns are year+quarter variables
    plt.clf()

<Figure size 432x288 with 0 Axes>

In [53]:
#Visualizations: BM, 5 lags, 2003-2019

quarter_list = ['Q1', 'Q2', 'Q3', 'Q4']

for i in range(22):
    plot = plt.errorbar(x=quarter_list, y=bm_data_quarterly_5lags_2003[i]['coefficient'], yerr = bm_data_quarterly_5lags_2003[i]['se'])
    plt.savefig(f'Data Visualizations/Corrected BEA, BM/BM, 5 lags, 2003/{bm_data.columns[i+2]}_quarterly_5lags.png', bbox_inches='tight') #i+2 since the first 2 columns are year+quarter variables
    plt.clf()

<Figure size 432x288 with 0 Axes>

In [66]:
#Visualizations: BM, 10 lags, 2003-2019

quarter_list = ['Q1', 'Q2', 'Q3', 'Q4']

for i in range(22):
    plot = plt.errorbar(x=quarter_list, y=bm_data_quarterly_10lags_2003[i]['coefficient'], yerr = bm_data_quarterly_10lags_2003[i]['se'])
    plt.savefig(f'Data Visualizations/Corrected BEA, BM/BM, 10 lags, 2003/{bm_data.columns[i+2]}_quarterly_10lags.png', bbox_inches='tight') #i+2 since the first 2 columns are year+quarter variables
    plt.clf()

<Figure size 432x288 with 0 Axes>

BEA + BM (5, 10 lags)

In [70]:
#Visualizations, BEA + BM (5 lags)

#List of Names to Compare

quarter_list=['Q1', 'Q2', 'Q3', 'Q4']
compare_names=[i for i in quarterly_per_change_names_bea[0][0:25] if i not in ['Quarterly % Change, Goods_2', 'Quarterly % Change, Goods_3', 'Quarterly % Change, Services_2', 'Quarterly % Change, Services_3', 'Quarterly % Change, Change in private inventories', 'Quarterly % Change, Net exports of goods and services']] #compare the 1st 26 observations, minus the import/export breakdown of goods + services. Only report aggregate goods/services
folder_names = {0: 'Table 8.1.3', 1: 'Table 8.1.4', 2: 'Table 8.1.5', 3: 'Table 8.1.6', 4: 'Table 8.1.11'}

for i in range(5): #Want Tables 8.1.3, 8.1.4, 8.1.5, 8.1.6, 8.1.11
    for j,k in zip(compare_names, range(22)): #use compare_names[0] since for some reason it's a nested list
        #set up if cases to compare. basically should just compare different bm_data_quarterly_5lags based on the first year of bea data
        #each entry of bea_data_quarterly_5lags is a dictionary with key given by quarterly % change name, value given by regression table
        #look at the first 5 entries of bea_data_quarterly_5lags. each corresponds to one of the tables. so compare the 1st index of each table 8.1.3, .., 8.1.11
        if sheet_names[i] in ['Table 8.1.3', 'Table 8.1.4', 'Table 8.1.6']: #First observation is 2002 for these tables
            y1, y2 = bea_data_quarterly_5lags_corrected[i][j]['coefficient'], bm_data_quarterly_5lags_2002[k]['coefficient']
            yerr1, yerr2 = bea_data_quarterly_5lags_corrected[i][j]['se'], bm_data_quarterly_5lags_2002[k]['se']
        elif sheet_names[i] == 'Table 8.1.5': #First observation is 1947
            y1, y2 = bea_data_quarterly_5lags_corrected[i][j]['coefficient'], bm_data_quarterly_5lags_1947[k]['coefficient']
            yerr1, yerr2 = bea_data_quarterly_5lags_corrected[i][j]['se'], bm_data_quarterly_5lags_1947[k]['se']
        else: #Table 8.1.11; first observation is 2003
            y1, y2 = bea_data_quarterly_5lags_corrected[i][j]['coefficient'], bm_data_quarterly_5lags_2003[k]['coefficient']
            yerr1, yerr2 = bea_data_quarterly_5lags_corrected[i][j]['se'], bm_data_quarterly_5lags_2003[k]['se']
        er1 = plt.errorbar(quarter_list, y1, yerr1, label='BEA')
        er2 = plt.errorbar(quarter_list, y2, yerr2, label='BM')
        plt.legend(loc='best')
        plt.savefig(f'Data Visualizations/Corrected BEA, BM/BEA + BM, 5 lags/{folder_names[i]}/{j[20:]}_quarterly_5lags.png', bbox_inches='tight') 
        plt.clf()

<Figure size 432x288 with 0 Axes>

In [71]:
#Visualizations, BEA + BM (10 lags)

#List of Names to Compare

quarter_list=['Q1', 'Q2', 'Q3', 'Q4']
compare_names=[i for i in quarterly_per_change_names_bea[0][0:25] if i not in ['Quarterly % Change, Goods_2', 'Quarterly % Change, Goods_3', 'Quarterly % Change, Services_2', 'Quarterly % Change, Services_3', 'Quarterly % Change, Change in private inventories', 'Quarterly % Change, Net exports of goods and services']] #compare the 1st 26 observations, minus the import/export breakdown of goods + services. Only report aggregate goods/services
folder_names = {0: 'Table 8.1.3', 1: 'Table 8.1.4', 2: 'Table 8.1.5', 3: 'Table 8.1.6', 4: 'Table 8.1.11'}

for i in range(5): #Want Tables 8.1.3, 8.1.4, 8.1.5, 8.1.6, 8.1.11
    for j,k in zip(compare_names, range(22)): #use compare_names[0] since for some reason it's a nested list
        #set up if cases to compare. basically should just compare different bm_data_quarterly_5lags based on the first year of bea data
        #each entry of bea_data_quarterly_5lags is a dictionary with key given by quarterly % change name, value given by regression table
        #look at the first 5 entries of bea_data_quarterly_5lags. each corresponds to one of the tables. so compare the 1st index of each table 8.1.3, .., 8.1.11
        if sheet_names[i] in ['Table 8.1.3', 'Table 8.1.4', 'Table 8.1.6']: #2002
            y1, y2 = bea_data_quarterly_10lags_corrected[i][j]['coefficient'], bm_data_quarterly_10lags_2002[k]['coefficient']
            yerr1, yerr2 = bea_data_quarterly_10lags_corrected[i][j]['se'], bm_data_quarterly_10lags_2002[k]['se']
        elif sheet_names[i] == 'Table 8.1.5': #1947
            y1, y2 = bea_data_quarterly_10lags_corrected[i][j]['coefficient'], bm_data_quarterly_10lags_1947[k]['coefficient']
            yerr1, yerr2 = bea_data_quarterly_10lags_corrected[i][j]['se'], bm_data_quarterly_10lags_1947[k]['se']
        else: #Table 8.1.11; 2003
            y1, y2 = bea_data_quarterly_10lags_corrected[i][j]['coefficient'], bm_data_quarterly_10lags_2003[k]['coefficient']
            yerr1, yerr2 = bea_data_quarterly_10lags_corrected[i][j]['se'], bm_data_quarterly_10lags_2003[k]['se']
        er1 = plt.errorbar(quarter_list, y1, yerr1, label='BEA')
        er2 = plt.errorbar(quarter_list, y2, yerr2, label='BM')
        plt.legend(loc='best')
        plt.savefig(f'Data Visualizations/Corrected BEA, BM/BEA + BM, 10 lags/{folder_names[i]}/{j[20:]}_quarterly_10lags.png', bbox_inches='tight') 
        plt.clf()

<Figure size 432x288 with 0 Axes>